# IR Lab WiSe 2023: Stopword Lists

This tutorial shows how to configure and use custom stopword lists in PyTerrier.

**Attention:** The scenario below is cherry-picked to explain the concept of stopword lists with a minimal example.


## Preparation: Install dependencies

In [1]:
# This is only needed in Google Colab, in a dev container, everything should be installed already
!pip3 install python-terrier

## Our Scenario

We want to build a search engine to support web developers working with CSS.

Our search engine has the following 3 documents:



In [1]:
documents = [
    {'docno': 'd1', 'text': 'In CSS, ::before creates a pseudo-element that is the first child of the selected element.'},
    {'docno': 'd2', 'text': 'In CSS, ::after creates a pseudo-element that is the last child of the selected element.'},
    {'docno': 'd3', 'text': 'The ::first-line CSS pseudo-element applies styles to the first line of a block-level element.'}
]

We create an index containing our three documents and use BM25 as retrieval model:

In [3]:
import pyterrier as pt
if not pt.started():
    pt.init()

def create_index(documents):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

index = create_index(documents)

In [4]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

## The Problem

Our search engine is now in operation for a while and we already received some positive feedback.
Still, we received some mails that complained that our search engine retrieves no relevant results for queries like "before", "css before", "after", and "CSS after".

We wonder why this is the case, because our index contains relevant documents for all four mentioned queries: (1) document `d1` is relevant for queries like "before" and "css before", and (2) document `d2` is relevant for queries like "after", and "CSS after".

Lets look into the problem:

In [5]:
# searching for before returns no results
bm25.search("before")

,docid,docno,rank,score,qid,query


In [6]:
# searching for after returns no results
bm25.search("after")

,docid,docno,rank,score,qid,query


In [7]:
# searching for css before returns results, but the relevant document d2 is only on the last position
bm25.search("css after")

,qid,docid,docno,rank,score,query
0,1,2,d3,0,-2.513562,css after
1,1,0,d1,1,-2.981605,css after
2,1,1,d2,2,-2.981605,css after


## The Solution

After some more debugging, we found out that [after](https://github.com/terrier-org/terrier-desktop/blob/master/share/stopword-list.txt#L206) and [before](https://github.com/terrier-org/terrier-desktop/blob/master/share/stopword-list.txt#L332) are both on the default stopword list and therefore removed from the documents and index.

To address this problem systematically, we create a small Cranfield-Style collection to measure if our bugfixes to the stopword list improve the retrieval:

In [8]:
# The information needs that we want to test
import pandas as pd

topics = pd.DataFrame([
    {'qid': '1', 'query': 'before'},
    {'qid': '2', 'query': 'css before'},
    {'qid': '3', 'query': 'after'},
    {'qid': '4', 'query': 'CSS after'},
])

qrels = pd.DataFrame([
    {'qid': '1', 'docno': 'd1', 'relevance': 1}, #d1 is the only relevant document for query 1
    {'qid': '2', 'docno': 'd1', 'relevance': 1}, #d1 is the only relevant document for query 2
    {'qid': '3', 'docno': 'd2', 'relevance': 1}, #d2 is the only relevant document for query 3
    {'qid': '3', 'docno': 'd2', 'relevance': 1}, #d2 is the only relevant document for query 3
])

In [9]:
pt.Experiment([bm25], topics, qrels, eval_metrics=['ndcg_cut_3', 'P_1'])

,name,ndcg_cut_3,P_1
0,BR(BM25),0.166667,0.0


Alright, now that we can measure the effectiveness, lets try to improve the effectiveness.

By thinking about the problem, we came to the conclusion that our stopword list should contain terms like "the", but also "css" should be a stopword because all our documents are on CSS.

To implement this, we store the terms the and css in a file called custom-stopwords.txt and we configure the `stopwords.filename` property of pyterrier so that our new stopword list is used.

In [10]:
!echo -e "the\ncss" > custom-stopwords.txt
#check the content of the stopword list:
!cat custom-stopwords.txt

the
css


In [11]:
# we use our new stopword list
pt.set_property("stopwords.filename", "./custom-stopwords.txt")

# we create a new index
index = create_index(documents)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [12]:
pt.Experiment([bm25], topics, qrels, eval_metrics=['ndcg_cut_3', 'P_1'])

,name,ndcg_cut_3,P_1
0,BR(BM25),1.0,1.0


# Summary

We made quite some improvement with adjusting our stopword list (nDCG@3 improved from 0.167 to 1.0 and Precision@1 improved from 0.0 to 1.0).

To summarize everything, please answer the following three questions:


### Question 1:

Is Stopword-removal a Precision-Oriented or a Recall-Oriented technique?


### TODO: Add your Solution

### Question 2:

There are many Different Stopword lists out there, please find 3 stopword lists and skim over them, do you spot obvious differences or surprising terms?


### TODO: Add your Solution


### Question 3:

Do you know famous phrases that are challenging to retrieve when we apply stopword removal?


### TODO: Add your Solution

### Question 4:

Given the task of building a domain-specific search engine, do you think ChatGPT would be useful to construct an effective stopword list? What would be the benefits, what would be the caveats? How would you use ChatGPT for that?


### TODO: Add your Solution